In [21]:
import warnings

import arviz as az
import matplotlib.pyplot as plt
import numpy as np
import pymc3 as pm
import theano.tensor as tt
import time
warnings.simplefilter(action="ignore", category=FutureWarning)
import sys,os

sys.path.append('../')
from src.utils.serialize import pickle_obj, unpickle_obj

In [2]:
def logistic(x):
    return 1/(1+np.exp(-x))

In [3]:
np.random.seed(0)
d = 2
groups = 1
N = 100



beta0 = 0.
betas = []
observations = []
covs = []

for g in range(groups):
    beta = beta0 + 4.*np.random.randn(d)
    betas.append(beta)
    
    cov = np.random.rand(d*N).reshape(N,d)*2.-1.
    covs.append(cov)
    p = logistic(cov.dot(beta))
    y = (np.random.random(len(p)) < p)+0.
    observations.append(y)

In [4]:
np.random.seed(0)
init_x = np.random.randn(4)

In [5]:
mu0 = np.array([0. for _ in range(d)])
sig0 = np.diag([1. for _ in range(d)])
mu1 = np.array([0. for _ in range(d)])
sig1 = sig0 + sig0
sig10 = sig0

In [22]:
mu0

array([0., 0.])

In [7]:
with pm.Model() as first_model:
    #priors on parameters
    mu_0 = pm.MvNormal('mu_0', mu=mu0, cov=sig0, shape=(2,))
    beta = pm.MvNormal('beta', mu=mu_0, cov=sig0, shape=(2,))
    

    
    #the probability of belonging to class 1
    p = pm.Deterministic('p', pm.math.sigmoid(beta[0]*covs[0][:,0]+
                               beta[1]*covs[0][:,1])
                        )

    #fit the data 
    observed=pm.Bernoulli("occupancy", p, observed=observations[0])
    
    start = pm.find_MAP()
    

/home/james/miniconda3/envs/bps/lib/python3.8/site-packages/theano/tensor/elemwise.py:826: RuntimeWarning: divide by zero encountered in log
  variables = ufunc(*ufunc_args, **ufunc_kwargs)
/home/james/miniconda3/envs/bps/lib/python3.8/site-packages/theano/tensor/elemwise.py:826: RuntimeWarning: invalid value encountered in multiply
  variables = ufunc(*ufunc_args, **ufunc_kwargs)


In [8]:
num_chains = 1

In [9]:
start

{'mu_0': array([2.13799408, 0.61352211]),
 'beta': array([4.27598974, 1.22704567]),
 'p': array([0.72888595, 0.42681323, 0.60532284, 0.97537811, 0.92862367,
        0.83567158, 0.00917811, 0.03602671, 0.96397438, 0.99205548,
        0.58880234, 0.05111885, 0.12359944, 0.49435499, 0.20742438,
        0.44841958, 0.02131891, 0.77653849, 0.98581282, 0.20488875,
        0.64814404, 0.86353267, 0.03268996, 0.12865325, 0.61055455,
        0.96083219, 0.03486255, 0.66905948, 0.28591396, 0.02037901,
        0.61040034, 0.05132045, 0.85275536, 0.86976545, 0.98197381,
        0.98706184, 0.71408388, 0.05789407, 0.06421903, 0.14591449,
        0.03714558, 0.49845467, 0.31051982, 0.8459294 , 0.24218191,
        0.06799079, 0.07053684, 0.39225504, 0.8316779 , 0.72237276,
        0.95873608, 0.1232766 , 0.72401258, 0.22147131, 0.59819162,
        0.77009265, 0.91905739, 0.9694877 , 0.97673086, 0.87320737,
        0.98598873, 0.4036953 , 0.0099636 , 0.70153792, 0.6972966 ,
        0.02627651, 0.69452

In [10]:
start = time.time()
with first_model:    
    #samples from posterior distribution 
    trace=pm.sample(2000, tune=100, cores=num_chains, chains=num_chains, compute_convergence_checks=False)
duration = time.time()-start
print(duration)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Sequential sampling (1 chains in 1 job)
NUTS: [beta, mu_0]


Sampling 1 chain for 100 tune and 2_000 draw iterations (100 + 2_000 draws total) took 132 seconds.
The acceptance probability does not match the target. It is 0.9334048915917911, but should be close to 0.8. Try to increase the number of tuning steps.


132.80891251564026


In [12]:
np.mean(trace.get_values('beta'))

2.8289971797214024

In [ ]:
np.std(trace.get_values('beta_2'))

In [ ]:
with first_model:
    az.plot_trace(trace)

In [13]:
df = az.summary(trace, round_to=2)

arviz - WARNING - Shape validation failed: input_shape: (1, 2000), minimum_shape: (chains=2, draws=4)


In [17]:
cols = [col for col in df.columns if 'ess' in col]

In [15]:
duration = 132

In [18]:
df['mean'][4:].min(), df['mean'][4:].max()

(0.01, 0.99)

In [19]:
df[cols]/duration/num_chains

,ess_mean,ess_sd,ess_bulk,ess_tail
mu_0[0],10.115758,9.233561,10.299015,11.449545
mu_0[1],6.880833,6.776288,6.897727,8.171818
beta[0],11.445076,11.249773,11.493788,11.039924
beta[1],8.577273,8.577273,8.537955,8.778258
p[0],9.014621,8.966742,9.007803,11.573561
...,...,...,...,...
p[95],16.100833,15.423030,15.467879,11.745152
p[96],12.324621,12.231970,11.785758,10.554242
p[97],9.224091,9.224091,8.265530,10.498485
p[98],13.158030,13.158030,10.467424,11.211894


In [ ]:
betas

In [23]:
first_model

In [25]:
pickle_obj(first_model, 'logistic.pmc3')

In [27]:
logistic_model = unpickle_obj('logistic.pmc3')

In [28]:
start = time.time()
with logistic_model:    
    #samples from posterior distribution 
    trace=pm.sample(2000, tune=100, cores=num_chains, chains=num_chains, compute_convergence_checks=False)
duration = time.time()-start
print(duration)

df = az.summary(trace, round_to=2)
df

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
/home/james/miniconda3/envs/bps/lib/python3.8/site-packages/theano/tensor/elemwise.py:826: RuntimeWarning: divide by zero encountered in log
  variables = ufunc(*ufunc_args, **ufunc_kwargs)
/home/james/miniconda3/envs/bps/lib/python3.8/site-packages/theano/tensor/elemwise.py:826: RuntimeWarning: invalid value encountered in multiply
  variables = ufunc(*ufunc_args, **ufunc_kwargs)
Sequential sampling (1 chains in 1 job)
NUTS: [beta, mu_0]


Sampling 1 chain for 100 tune and 2_000 draw iterations (100 + 2_000 draws total) took 106 seconds.
The acceptance probability does not match the target. It is 0.9052641853225213, but should be close to 0.8. Try to increase the number of tuning steps.


107.0102128982544


arviz - WARNING - Shape validation failed: input_shape: (1, 2000), minimum_shape: (chains=2, draws=4)


,mean,sd,hdi_3%,hdi_97%,mcse_mean,mcse_sd,ess_mean,ess_sd,ess_bulk,ess_tail,r_hat
mu_0[0],2.19,0.82,0.58,3.61,0.02,0.02,1214.93,1214.93,1215.08,1007.80,NaN
mu_0[1],0.60,0.78,-1.03,1.89,0.02,0.02,1301.98,1200.37,1299.32,1100.63,NaN
beta[0],4.37,0.71,2.91,5.62,0.02,0.01,1693.44,1682.09,1701.83,1604.28,NaN
beta[1],1.25,0.51,0.36,2.27,0.02,0.01,1118.95,1104.92,1118.22,1229.88,NaN
p[0],0.73,0.03,0.66,0.79,0.00,0.00,1330.71,1330.71,1335.93,1452.20,NaN
...,...,...,...,...,...,...,...,...,...,...,...
p[95],0.17,0.07,0.05,0.30,0.00,0.00,2100.33,1988.87,2111.93,1683.04,NaN
p[96],0.88,0.03,0.82,0.95,0.00,0.00,1659.39,1659.39,1731.97,1401.31,NaN
p[97],0.05,0.03,0.01,0.10,0.00,0.00,1181.07,1181.07,1151.41,1408.29,NaN
p[98],0.02,0.01,0.00,0.05,0.00,0.00,1456.50,1456.50,1561.27,1605.56,NaN
